# Clasificadores Bayesianos

## Caso: Titanic

En el hundimiento del Titanic murieron 1514 personas de las 2223 que iban a bordo, lo que convierte a esta tragedia en uno de los mayores naufragios de la historia. Vamos a utilizar un dataset (csv, separado por compas) el cual contiene un listado de 1309 pasajeros que estuvieron a bordo del Titanic para analizar la supervivencia de los pasajeros según ciertas caracteristicas (sexo, edad, ticket de clase, entre otras).

## Análisis Exploratorio y Descriptivo

*   Instalar todas las librerias python que aquí se incluyen.
*   Explorar los datos con Pandas, ordenándolos o graficándolos.


In [ ]:
# Importamos las librerías que necesitamos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pydotplus
import pickle
from sklearn.model_selection import GridSearchCV

# Matriz de confusión
from sklearn.metrics import confusion_matrix

# Library & dataset
import seaborn as sns


In [ ]:
# Cargamos el dataset
df = pd.read_csv('https://raw.githubusercontent.com/PabloSoligo2014/3670-UNLaM-CD/refs/heads/main/datasets/titanic.csv', sep = ',')

# No vamos a utilizar el PassengerId,Name porque son de tipo ID que no tienen ningun uso como variables predictoras para un modelo de clasificacion
df.drop(['PassengerId', 'Name'], axis=1, inplace=True)

# Por lo pronto vamos a quitar las variables Ticket/Cabin que son variables categoricas de alta dimensionalidad
# en una clase a posteriori vamos a buscar la mejor forma de tratar estas variables para ser usadas en metodos de clasificacion
df.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

df.head(10)

Siempre el primer paso para analizar cualquier problema de clasificacion, es analizar la distribucion de la variable target.

In [ ]:
df['Survived'].value_counts()

Analicemos con un gráfico de barras, la relación entre los clientes que entran en mora de los que no, según los siguientes aspectos:

*   Por tipo de empleo

In [ ]:
pd.crosstab(index=df['Sex'],
            columns=df['Survived'],
            margins=False).plot.barh(stacked=True,)

Esto mismo podríamos verlo con una tabla de doble entrada:

In [ ]:
pd.crosstab(index=df['Sex'],
            columns=df['Survived'],
            margins=False)


*   Por clase del boleto (1ra, 2da, etc).

In [ ]:
pd.crosstab(index=df['Pclass'],
            columns=df['Survived'],
            margins=False).plot.barh(stacked=True,)

*   Por cantidad de hermanxs

In [ ]:
pd.crosstab(index=df['SibSp'],
            columns=df['Survived'],
            margins=False).plot.barh(stacked=True,)

¿Cómo es la distribución de las edades de los clientes?

In [ ]:
# Construimos un gráfico de densidad
plt.title('Age')
sns.kdeplot(df['Age'], shade=True) # shade indica si el gráfico es sombreado o no

In [ ]:
val_nulos = df.isnull().sum()
print(val_nulos)

#### **Transformaciones**

Encodeamos como booleanos todos los atributos categóricos a utilizar (sin incluir la variable clase). Para ello, aplicamos el método `pd.get_dummies`.

In [ ]:
df.head(3)

In [ ]:
# Armamos las variables predictoras para el algoritmo
X = pd.get_dummies(df.drop('Survived', axis=1))
atributos = X.columns

X.head()

Encodeamos las etiquetas del atributo clase usando el método `LabelEncoder`.  Este método convierte una variable categórica a numérica, con valores entre "*0 and nro_clases-1*", para simplificar los cálculos que hace el algoritmo.

En nuestro caso los valores para el atributo clase serán 0 (MORA) y 1 (NO MORA).

In [ ]:
# Separamos la variable target en un atributo generico y
y = df['Survived']
y[:10]

#### **Partición del conjunto de datos**

Dividimos el dataset original en dos conjuntos de datos:

*  Conjunto de entrenamiento (70%)
*  Conjunto de prueba (30%)

In [ ]:
# Importamos la librería que necesitamos
from sklearn.model_selection import train_test_split

# Dado que el clasificador Naive Bayes no puede trabajar con valores numericos nulos, vamos a asignar un valor default de 0
# Como se ya se vio en clases previas, este no es el camino y hay que hacer un apropiado analysis/tratamiento de valoes nulos para cada variable
# y definir la estrategia a utilizar
X.fillna(0, inplace=True)

# Dividimos X e y con train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"X: {X.shape} - X_train:{X_train.shape} - X_test:{X_test.shape}")

## B. Clasificador Naïve Bayes

En esta parte aprenderemos cómo aplicar un modelo **Naïve Bayes** a problemas de clasificación. Para ello, utilizaremos el dataset de clientes anterior y construiremos un modelo para predecir el mismo objetivo, es decir, si un cliente va a caer en mora en algun momento de la vida del mismo.

#### Referencia: [Documentación de Naïve Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)

#### **Aplicación del algoritmo de Naïve Bayes**

Con los datos ya particionados vamos a entrenar dos modelos utilizando dos variantes del algoritmo Naïve Bayes, el multinomial y el gausiano.  

## 1. Construcción y evaluación del primer modelo

Para este modelo usaremos el algoritmo ***Naive Bayes “multinomial”***. Este clasificador es adecuado cuando se tienen características discretas, siendo uno de los algoritmos estándar usado por ejemplo para clasificación o categorización de texto.


In [ ]:
# Importamos la librería que necesitamos
from sklearn.naive_bayes import MultinomialNB # naive bayes multinomial para clasificación

# Creamos y entrenamos el clasificador bayesiano
bayes_multi = MultinomialNB()
bayes_multi.fit(X_train, y_train) # entrenamos el clasificador

Como ejemplo, podemos guardar el modelo entrenado en un archivo para volver a cargarlo cuando necesitemos clasificar registros nuevos

In [ ]:
print("Antes del dump el objeto era: ")
print(bayes_multi)

# Dejo espacio para no confundir
print(" "*30)
print("-"*30)
print(" "*30)

# Guardo el objeto en un archivo llamado "modelo.bayes_multi"
pickle.dump(bayes_multi, open('modelo.bayes_multi', 'wb'))

# Ahora levanto el objeto desde el archivo guardado, de nuevo en la misma variable dic
bayes_multi = pickle.load(open('modelo.bayes_multi', 'rb'))

print("Luego del load el objeto es: ")
print(bayes_multi)

Ahora que nuestro primer modelo ha sido entrenado, podemos utilizar el *conjunto de prueba* para verificar su capacidad de predicción.

In [ ]:
# Calculamos y mostramos la matriz de confusión del modelo
y_pred_multi = bayes_multi.predict(X_test)
conf = confusion_matrix(y_test, y_pred_multi)

predicted_cols = ['pred_'+str(c) for c in y.unique()]
real_cols = ['real_'+str(c) for c in y.unique()]
pd.DataFrame(conf, index=real_cols, columns = predicted_cols)

Calculamos las metricas para evaluación de modelos de clasificación automatizadas.

Recordemos que:
- **Exactitud (Accuracy)** = TP+TN / (TP + TN + FP + FN)           
- **Error de Predicción** = 1 - Exactitud
- **Precisión** = TP / (TP + FP)
- **Sensibilidad (Recall)** = TP / (TP + FN)
<br>
<br>

In [ ]:
# Reporte del clasificador
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_multi))

## 2. Construcción y evaluación del segundo modelo
Para el segundo modelo, probaremos con el algoritmo ***Naive Bayes "gausiano"***. Este algoritmo es más adecuado para datos continuos ya que asume que los datos tienen una distribución de curva de Gauss (normal).

Entrenamos el nuevo modelo propuesto para este caso:

In [ ]:
# Importamos la librería que necesitamos
from sklearn.naive_bayes import GaussianNB # naive bayes multinomial para clasificación

bayes_gauss = GaussianNB()
bayes_gauss.fit(X_train, y_train) # entrenamos el clasificador

Con el segundo modelo ya entrenado, utilizamos el *conjunto de prueba* para verificar su capacidad de predicción.

In [ ]:
# Calculamos y mostramos la matriz de confusión del modelo
y_pred_gauss = bayes_gauss.predict(X_test)
conf = confusion_matrix(y_test, y_pred_gauss)

predicted_cols = ['pred_'+str(c) for c in y.unique()]
real_cols = ['real_'+str(c) for c in y.unique()]
pd.DataFrame(conf, index=real_cols, columns = predicted_cols)

In [ ]:
# Reporte del clasificador
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_gauss))

## C: Optimización con GridSearchCV

### Modelo Naïve Bayes

Los clasificadores Naïve Bayes ofrecen diversos hiper-parámetros para cada uno de los algoritmos que incluyen. Vamos a utilizar los parámetros para el algoritmo Gaussiano, utilizado en el primer modelo, con el cual se obtuvieron mejores estimaciones.

In [ ]:
# Realizamos la búsqueda con Grid Search para el modelo Gaussiano
model = GaussianNB() # modelo a utilizar
parametros = {'var_smoothing': np.logspace(0, -9, num=100)}

# Tambien podrian probar multinomial cuyo hyperparametro es alpha
# model = MultinomialNB() # modelo a utilizar
# parametros = {'alpha':[0.0001, 0.001, 0.05, 0.1, 1]}

# En este caso vamos a buscar la mejor configuracion del modelo para optimizar la metrica **precision** pueden elegir la que mejor aplique a su problema
gs = GridSearchCV(model, parametros, scoring='precision', verbose=1 , n_jobs=-1)
gs.fit(X, y)

**Nota**: El hiper-parámetro ***var_smoothing*** se utiliza para realizar un suavizado en la distribucion de las variables predictoras, aplicando un filtro en comparacion con la distribucion gaussiana. Es un metodo de regularizacion para entrenar un model mas estable y reducir el overfitting.
- siempre podemos validar los hiperparametros de cada modelo en la documentacion de scikit learn: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

**Nota**: El hiper-parámetro ***alpha*** se utiliza para establecer el suavizado de Laplace, para que la probabilidad resultante nunca sea cero y al menos el dato aparezca una vez. Valores mayores fomentan el underfitting.

Mostramos los mejores resultados obtenidos a partir de los hiper-parámetros utilizados.

In [ ]:
print(gs.best_estimator_)
print(gs.best_score_)

Utilizando el *conjunto de prueba* y los mejores hiper-parámetros seleccionados, verificamos la capacidad de predicción del modelo obtenido.

In [ ]:
# Calculamos y mostramos la matriz de confusión del modelo
y_pred = gs.best_estimator_.predict(X_test)
conf = confusion_matrix(y_test, y_pred)

predicted_cols = ['pred_'+str(c) for c in y.unique()]
real_cols = ['real_'+str(c) for c in y.unique()]
pd.DataFrame(conf, index=real_cols, columns = predicted_cols)

In [ ]:
# Reporte del clasificador
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))